In [1]:
import time
from pathlib import Path
import numpy as np
import pandas as pd
import tTEM_toolbox as tt
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import plotly.io as pio
pio.renderers.default = "browser"
pio.kaleido.scope.chromium_args = tuple([arg for arg in pio.kaleido.scope.chromium_args if arg != "--disable-dev-shm-usage"])
workdir = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test')
welllog = workdir.joinpath(r'Plot_with_well_log\Well_log.xlsx')
elevation = workdir.joinpath(r'well_Utah\usgs_water_elevation.csv')
ttemname_north = workdir.joinpath(r'Plot_with_well_log\PD1_I01_MOD.xyz')
ttemname_center = workdir.joinpath(r'Plot_with_well_log\PD22_I03_MOD.xyz')
ttem_lslake = workdir.joinpath(r'Plot_with_well_log\lsll_I05_MOD.xyz')
gamma_bradshaw = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\Gamma_data\bradshaw farms down.csv'
gamma_adams = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\Gamma_data\adams down.csv'
gamma_halterman = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\halterman down.csv'
gamma_location = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\location.csv'
DOI = workdir.joinpath(r'Plot_with_well_log\DOID1_DOIStaE.xyz')
well_info = workdir.joinpath(r'well_Utah\USGSdownload\NWISMapperExport.xlsx')
colorRes = [[0, 'rgb(0,0,190)'],
            [1 / 16, 'rgb(0,75,220)'],
            [2 / 16, 'rgb(0,150,235)'],
            [3 / 16, 'rgb(0,200,255)'],
            [4 / 16, 'rgb(80,240,255)'],
            [5 / 16, 'rgb(30,210,0)'],
            [6 / 16, 'rgb(180,255,30)'],
            [7 / 16, 'rgb(255,255,0)'],
            [8 / 16, 'rgb(255,195,0)'],
            [9 / 16, 'rgb(255,115,0)'],
            [10 / 16, 'rgb(255,0,0)'],
            [11 / 16, 'rgb(255,0,120)'],
            [12 / 16, 'rgb(140,40,180)'],
            [13 / 16, 'rgb(165,70,220)'],
            [14 / 16, 'rgb(195,130,240)'],
            [15 / 16, 'rgb(230,155,240)'],
            [1, 'rgb(230,155,255)']]
def distance_of_two_points(x1:float,y1:float,x2:float,y2:float):
    distance = np.sqrt((x1-x2)**2+(y1-y2)**2)
    return distance
def split_ttem(ttem_df, gwsurface_result):
    def get_distance(group1, group2):
        dis = np.sqrt((group1[0] - group2[0]) ** 2 + (group1[1] - group2[1]) ** 2)
        return dis
    abv_water_table = []
    blw_water_table = []
    ttem_groups = ttem_df.groupby(['UTMX', 'UTMY'])
    well_location = gwsurface_result[['UTMX', 'UTMY']].values
    for name, group in ttem_groups:
        ttem_xy = list(group[['UTMX', 'UTMY']].iloc[0])
        ttem_well_distance = list(map(lambda x: get_distance(ttem_xy, x), well_location))
        match = gwsurface_result.iloc[ttem_well_distance.index(min(ttem_well_distance))]
        elevation = match['water_elevation']
        ttem_abv = group[group['Elevation_End'] >= elevation]
        abv_water_table.append(ttem_abv)
        ttem_blw = group[group['Elevation_End'] < elevation]
        blw_water_table.append(ttem_blw)

    ttem_above = pd.concat(abv_water_table)
    ttem_below = pd.concat(blw_water_table)
    return ttem_above, ttem_below
def plot_bootstrap_result(dataframe):
    """
    plot bootstrap result

    :param dataframe:
    :return: plotly fig
    """
    fig_hist = go.Figure()
    fig_hist.data = []
    fig_hist.add_trace(go.Histogram(x=dataframe.fine, name='Fine', marker_color='Blue', opacity=0.75))
    fig_hist.add_trace(go.Histogram(x=dataframe.coarse, name='Coarse', marker_color='Red', opacity=0.75))
    if dataframe.mix.sum() == 0:
        print("skip plot mix because there is no data")
    else:
        fig_hist.add_trace(go.Histogram(x=dataframe.mix, name='Mix', marker_color='Green', opacity=0.75))
    fig_hist.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')

    return fig_hist
def data_fence(ttem_df,xmin,ymin,xmax,ymax):
    if xmin>xmax:
        raise ValueError('xmin:{} is greater than xmax:{}'.format(xmin,xmax))
    if ymin>ymax:
        raise ValueError('ymin:{} is greater than ymax:{}'.format(xmin,xmax))
    new_ttem_df = ttem_df[(ttem_df['UTMX']>xmin)&(ttem_df['UTMX']<xmax)&(ttem_df['UTMY']>ymin)&(ttem_df['UTMY']<ymax)]
    return new_ttem_df
def filter_line(ttem_df, line_filter):
        ttem_preprocess_line = ttem_df[ttem_df['Line_No'] == int(line_filter)]
        if ttem_preprocess_line.empty:
            raise ValueError('Did not found any data under line_no {}, the line number suppose to be integer'.format(line_filter))
        return ttem_preprocess_line
def resistivity_avg(ttem_df):
    new_1d_df = []
    coordinate_group = ttem_df.groupby(['UTMX','UTMY'])
    for name, group in coordinate_group:
        mean_resistivity = group['Resistivity'].mean()
        output_df = {'UTMX':name[0],'UTMY':name[1],'Mean_Resistivity':mean_resistivity}
        tmp_df = pd.DataFrame([output_df],index=None)
        new_1d_df.append(tmp_df)
    export_df = pd.concat(new_1d_df)
    return  export_df
def lithology_pct(rock_transform_df):
    """
    # Plan to group by the coordinate and generate a new dataframe with only [UTMX, UTMY]
    """
    new_1d_df = []
    coordinate_group = rock_transform_df.groupby(['UTMX','UTMY'])
    for name, group in coordinate_group:
        total_thickness_for_point = sum(group['Thickness'])
        lithology_group = group.groupby('Identity')
        try:
            fine_grain_thickness = lithology_group.get_group('Fine_grain')['Thickness'].sum()
        except KeyError:
            fine_grain_thickness = 0
        try:
            mixed_grain_thickness = lithology_group.get_group('Mix_grain')['Thickness'].sum()
        except KeyError:
            mixed_grain_thickness = 0
        try:
            coarse_grain_thickness = lithology_group.get_group('Coarse_grain')['Thickness'].sum()
        except KeyError:
            coarse_grain_thickness = 0
        tmp_df = pd.DataFrame([{'UTMX':name[0],
                  'UTMY':name[1],
                  'Fine grained ratio':fine_grain_thickness/float(total_thickness_for_point),
                  'mixed_grain_thickness':mixed_grain_thickness/float(total_thickness_for_point),
                  'coarse_grain_thickness':coarse_grain_thickness/float(total_thickness_for_point)}])

        new_1d_df.append(tmp_df)
    export = pd.concat(new_1d_df)
    return export
def block_plot(ttem_df, line_filter=None, fence=None, plot_ft=True):

    """
    This is the function that plot the given dataframe into gridded plot (for better looking)

    :param DataFrame ttem_df: a data frame contains tTEM data exported from Aarhus workbench
    :param int line_filter: In tTEM dataframe you can filter tTEM data by Line_No (default: None)
    :param list fence: receive a UTM coordinate to cut data, ex: [xmin, ymin, xmax, ymax] (default: None)
    :param plot_ft: True: plot all parameter in feet, False: plot in meter (default: True)
    :return:
        - fig - plotly fig of the block_plot
        - ttem_for_plot - the tTEM dataframe use for plot
        - empty_grid - the grid use to plot the figure
    """
    if line_filter is not None and fence is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocess_fence = data_fence(ttem_preprocess_line,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    elif line_filter is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocessed = ttem_preprocess_line.copy()
    elif fence is not None:
        ttem_preprocess_fence = data_fence(ttem_df,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    else:
        ttem_preprocessed = ttem_df
    if plot_ft is True:
        m_to_ft_factor = 3.28
        unit = 'ft'
    else:
        m_to_ft_factor = 1
        unit = 'm'
    distance=0
    ttem_coordinate_group = ttem_preprocessed.groupby(['UTMX','UTMY'])
    for i in range(1,len(list(ttem_coordinate_group.groups.keys()))):
        tmp =distance_of_two_points(list(ttem_coordinate_group.groups.keys())[i-1][0],
                                     list(ttem_coordinate_group.groups.keys())[i-1][1],
                                     list(ttem_coordinate_group.groups.keys())[i][0],
                                     list(ttem_coordinate_group.groups.keys())[i][1])
        distance=distance+tmp
    distance_range = int(distance*m_to_ft_factor)
    ttem_preprocessed['Elevation_Cell'] = ttem_preprocessed['Elevation_Cell']*m_to_ft_factor
    ttem_preprocessed['Elevation_End'] = ttem_preprocessed['Elevation_End']*m_to_ft_factor
    elevation_max = ttem_preprocessed['Elevation_Cell'].max()

    elevation_end_min = ttem_preprocessed['Elevation_End'].min()

    elevation_range = int(elevation_max-elevation_end_min)
    print('elevation_range',elevation_max,elevation_end_min,elevation_range)
    ttem_preprocessed['Elevation_Cell_for_plot'] = abs(ttem_preprocessed['Elevation_Cell'] - elevation_max)
    ttem_preprocessed['Elevation_End_for_plot'] = abs(ttem_preprocessed['Elevation_End']-ttem_preprocessed['Elevation_End'].max())
    y_distance = elevation_range
    x_distance = distance_range/10
    empty_grid = np.full((int((y_distance+500)),int(x_distance)),np.nan)
# Fill in the tTEM data by loop through the grid
    first_iteration = True
    for name, group in ttem_coordinate_group:
        if first_iteration is True:
            first_iteration = False
            initial_distance = 1
            tmp_coor = name
            for index, line in group.iterrows():
                elevation_cell_round = line['Elevation_Cell_for_plot']
                elevation_end_round = line['Elevation_End_for_plot']
                empty_grid[int((elevation_cell_round-6)*m_to_ft_factor):int((elevation_end_round+6)*m_to_ft_factor),int((initial_distance-10)*m_to_ft_factor/10):int(((initial_distance+10)*m_to_ft_factor/10))] = np.log10(line['Resistivity'])
        else:
            initial_distance = initial_distance + distance_of_two_points(tmp_coor[0],tmp_coor[1],name[0],name[1])
            tmp_coor = name
            for index, line in group.iterrows():
                elevation_cell_round = line['Elevation_Cell_for_plot']
                empty_grid[int((elevation_cell_round-6)*m_to_ft_factor):int((elevation_end_round+6)*m_to_ft_factor),int((initial_distance-10)*m_to_ft_factor/10):int(((initial_distance+10)*m_to_ft_factor/10))] = np.log10(line['Resistivity'])
    fig = px.imshow(empty_grid, range_color=(0,3),color_continuous_scale=colorRes)
    fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
    fig.update_layout(
        yaxis=dict(
        title='Elevation ({})'.format(unit),
        #tickmode='linear',
        #tick0=1774,
        #dtick=100
    ),
        xaxis=dict(
        title='Distance ({})'.format(unit)
    )
    )
    fig.update_layout(
        dict(
        xaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            )
        ),
        yaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            ),
            #tickmode='array',
            #tickvals=list(np.arange(0,empty_grid.shape[0],500)),
            #TODO: this need more modification for m scale
            #ticktext=[str(int(elevation_max-i/10)) for i in np.arange(0,empty_grid.shape[0],500)]
        ),)
    )
    fig.update_coloraxes(
        colorbar=dict(
            ticks='outside',
            title='Resistivity',
            tickvals=[0,1,2,3],
            tickfont=dict(
                size=30
            ),
            ticktext=['1','10','100','1000'],
            tickmode='array',
            len=0.5))
    """fig.update_layout(
        title=dict(
            text='tTEM Line100, Northern Parowan Valley',
        ),
        titlefont=dict(
            size=50
        )
    )"""
    fig.show(renderer='browser')
    return fig, ttem_preprocessed,empty_grid
def block_plot2(ttem_df, line_filter=None, fence=None, plot_ft=True):

    """
    This is the function that plot the given dataframe into gridded plot (for better looking)

    :param DataFrame ttem_df: a data frame contains tTEM data exported from Aarhus workbench
    :param int line_filter: In tTEM dataframe you can filter tTEM data by Line_No (default: None)
    :param list fence: receive a UTM coordinate to cut data, ex: [xmin, ymin, xmax, ymax] (default: None)
    :param plot_ft: True: plot all parameter in feet, False: plot in meter (default: True)
    :return:
        - fig - plotly fig of the block_plot
        - ttem_for_plot - the tTEM dataframe use for plot
        - empty_grid - the grid use to plot the figure
    """
    if line_filter is not None and fence is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocess_fence = data_fence(ttem_preprocess_line,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    elif line_filter is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocessed = ttem_preprocess_line.copy()
    elif fence is not None:
        ttem_preprocess_fence = data_fence(ttem_df,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    else:
        ttem_preprocessed = ttem_df
    if plot_ft is True:
        m_to_ft_factor = 3.28
        unit = 'ft'
    else:
        m_to_ft_factor = 1
        unit = 'm'
    distance=0
    ttem_coordinate_group = ttem_preprocessed.groupby(['UTMX','UTMY'])
    for i in range(1,len(list(ttem_coordinate_group.groups.keys()))):
        tmp =distance_of_two_points(list(ttem_coordinate_group.groups.keys())[i-1][0],
                                     list(ttem_coordinate_group.groups.keys())[i-1][1],
                                     list(ttem_coordinate_group.groups.keys())[i][0],
                                     list(ttem_coordinate_group.groups.keys())[i][1])
        distance=distance+tmp
    distance_range = int(distance*m_to_ft_factor)
    ttem_preprocessed['Elevation_Cell'] = ttem_preprocessed['Elevation_Cell']*m_to_ft_factor
    ttem_preprocessed['Elevation_End'] = ttem_preprocessed['Elevation_End']*m_to_ft_factor
    elevation_max = ttem_preprocessed['Elevation_Cell'].max()

    elevation_end_min = ttem_preprocessed['Elevation_End'].min()

    elevation_range = int(elevation_max-elevation_end_min)
    print('elevation_range',elevation_max,elevation_end_min,elevation_range)
    ttem_preprocessed['Elevation_Cell_for_plot'] = abs(ttem_preprocessed['Elevation_Cell'] - elevation_max)
    ttem_preprocessed['Elevation_End_for_plot'] = abs(ttem_preprocessed['Elevation_End']-ttem_preprocessed['Elevation_End'].max())
    y_distance = elevation_range
    x_distance = distance_range/10
    empty_grid = np.full((int((y_distance+500)),int(x_distance)),np.nan)
# Fill in the tTEM data by loop through the grid
    first_iteration = True
    for name, group in ttem_coordinate_group:
        if first_iteration is True:
            first_iteration = False
            initial_distance = 1
            tmp_coor = name
            for index, line in group.iterrows():
                elevation_cell_round = line['Elevation_Cell_for_plot']
                elevation_end_round = line['Elevation_End_for_plot']
                empty_grid[int((elevation_cell_round-6)*m_to_ft_factor):int((elevation_end_round+6)*m_to_ft_factor),int((initial_distance-10)*m_to_ft_factor/10):int(((initial_distance+10)*m_to_ft_factor/10))] = np.log10(line['Resistivity'])
        else:
            initial_distance = initial_distance + distance_of_two_points(tmp_coor[0],tmp_coor[1],name[0],name[1])
            tmp_coor = name
            for index, line in group.iterrows():
                elevation_cell_round = line['Elevation_Cell_for_plot']
                empty_grid[int((elevation_cell_round-6)*m_to_ft_factor):int((elevation_end_round+6)*m_to_ft_factor),int((initial_distance-10)*m_to_ft_factor/10):int(((initial_distance+10)*m_to_ft_factor/10))] = np.log10(line['Resistivity'])
    fig = px.imshow(empty_grid, range_color=(0,3),color_continuous_scale=colorRes)
    fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
    fig.update_layout(
        yaxis=dict(
        title='Elevation ({})'.format(unit),
        #tickmode='linear',
        #tick0=1774,
        #dtick=100
    ),
        xaxis=dict(
        title='Distance ({})'.format(unit)
    )
    )
    fig.update_layout(
        dict(
        xaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            )
        ),
        yaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            ),
            #tickmode='array',
            #tickvals=list(np.arange(0,empty_grid.shape[0],500)),
            #TODO: this need more modification for m scale
            #ticktext=[str(int(elevation_max-i/10)) for i in np.arange(0,empty_grid.shape[0],500)]
        ),)
    )
    fig.update_coloraxes(
        colorbar=dict(
            ticks='outside',
            title='Resistivity',
            tickvals=[0,1,2,3],
            tickfont=dict(
                size=30
            ),
            ticktext=['1','10','100','1000'],
            tickmode='array',
            len=0.5))
    """fig.update_layout(
        title=dict(
            text='tTEM Line100, Northern Parowan Valley',
        ),
        titlefont=dict(
            size=50
        )
    )"""
    fig.show(renderer='browser')
    return fig, ttem_preprocessed,empty_grid
def block_plot_ID(ttem_df, line_filter=None, fence=None, plot_ft=True):

    """
    This is the function that plot the given dataframe into gridded plot (for better looking)

    :param DataFrame ttem_df: a data frame contains tTEM data exported from Aarhus workbench
    :param int line_filter: In tTEM dataframe you can filter tTEM data by Line_No (default: None)
    :param list fence: receive a UTM coordinate to cut data, ex: [xmin, ymin, xmax, ymax] (default: None)
    :param plot_ft: True: plot all parameter in feet, False: plot in meter (default: True)
    :return:
        - fig - plotly fig of the block_plot
        - ttem_for_plot - the tTEM dataframe use for plot
        - empty_grid - the grid use to plot the figure
    """
    if line_filter is not None and fence is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocess_fence = data_fence(ttem_preprocess_line,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    elif line_filter is not None:
        ttem_preprocess_line = filter_line(ttem_df,line_filter)
        ttem_preprocessed = ttem_preprocess_line.copy()
    elif fence is not None:
        ttem_preprocess_fence = data_fence(ttem_df,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    else:
        ttem_preprocessed = ttem_df
    if plot_ft is True:
        m_to_ft_factor = 3.28
        unit = 'ft'
    else:
        m_to_ft_factor = 1
        unit = 'm'
    ttem_preprocessed['Elevation_Cell'] = ttem_preprocessed['Elevation_Cell']*m_to_ft_factor
    ttem_preprocessed['Elevation_End'] = ttem_preprocessed['Elevation_End']*m_to_ft_factor
    elevation_max = ttem_preprocessed['Elevation_Cell'].max()
    elevation_end_min = ttem_preprocessed['Elevation_End'].min()
    elevation_range = int(elevation_max-elevation_end_min)
    #print('elevation_range',elevation_max,elevation_end_min,elevation_range)
    ttem_preprocessed['Elevation_Cell_for_plot'] = abs(ttem_preprocessed['Elevation_Cell'] - elevation_max)
    ttem_preprocessed['Elevation_End_for_plot'] = abs(ttem_preprocessed['Elevation_End']-ttem_preprocessed['Elevation_End'].max())
    distance_range = ttem_preprocessed['ID'].max() - ttem_preprocessed['ID'].min()
    y_distance = elevation_range
    x_distance = distance_range
    empty_grid = np.full((int((y_distance)),int(x_distance)),np.nan)
# Fill in the tTEM data by loop through the grid
    first_iteration = True
    ttem_ID_group = ttem_preprocessed.groupby('ID')
    for name,group in ttem_ID_group:
        for index, line in group.iterrows():
            empty_grid[int(line['Elevation_Cell_for_plot']-6*m_to_ft_factor):int(line['Elevation_End_for_plot']+6*m_to_ft_factor), int(name)-1:int(name)+1] = np.log10(line['Resistivity'])
    fig = px.imshow(empty_grid, range_color=(0,3),color_continuous_scale=colorRes)
    fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
    fig.update_layout(
        yaxis=dict(
        title='Elevation ({})'.format(unit),
        #tickmode='linear',
        #tick0=1774,
        #dtick=100
    ),
        xaxis=dict(
        title='Distance ({})'.format(unit)
    )
    )
    fig.update_layout(
        dict(
        xaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            )
        ),
        yaxis=dict(
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            ),
            #tickmode='array',
            #tickvals=list(np.arange(0,empty_grid.shape[0],500)),
            #TODO: this need more modification for m scale
            #ticktext=[str(int(elevation_max-i/10)) for i in np.arange(0,empty_grid.shape[0],500)]
        ),)
    )
    fig.update_coloraxes(
        colorbar=dict(
            ticks='outside',
            title='Resistivity',
            tickvals=[0,1,2,3],
            tickfont=dict(
                size=30
            ),
            ticktext=['1','10','100','1000'],
            tickmode='array',
            len=0.5))
    """fig.update_layout(
        title=dict(
            text='tTEM Line100, Northern Parowan Valley',
        ),
        titlefont=dict(
            size=50
        )
    )"""
    fig.show(renderer='browser')
    return fig, ttem_preprocessed,empty_grid
def rock_plot(rock_trans_df, line_filter=None, fence=None, plot_ft=True):

    """
    This is the function that plot the given dataframe into gridded plot (for better looking)

    :param ttem_df: a data frame contains tTEM data exported from Aarhus workbench
    :param line_filter: In tTEM dataframe you can filter tTEM data by Line_No (default: None)
    :param fence: receive a UTM coordinate to cut data, ex: [xmin, ymin, xmax, ymax] (default: None)
    :param plot_ft: True: plot all parameter in feet, False: plot in meter (default: True)
    :return:
    """
    if line_filter is not None and fence is not None:
        ttem_preprocess_line = filter_line(rock_trans_df,line_filter)
        ttem_preprocess_fence = data_fence(ttem_preprocess_line,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    elif line_filter is not None:
        ttem_preprocess_line = filter_line(rock_trans_df,line_filter)
        ttem_preprocessed = ttem_preprocess_line.copy()
    elif fence is not None:
        ttem_preprocess_fence = data_fence(rock_trans_df,fence[0],fence[1],fence[2],fence[3])
        ttem_preprocessed = ttem_preprocess_fence.copy()
    else:
        ttem_preprocessed = rock_trans_df
    if plot_ft is True:
        m_to_ft_factor = 3.28
        unit = 'ft'
    else:
        m_to_ft_factor = 1
        unit = 'm'
    ttem_preprocessed['distance'] = np.sqrt(ttem_preprocessed['UTMX']**2 + ttem_preprocessed['UTMY']**2)*m_to_ft_factor
    ttem_preprocessed['Elevation_Cell'] = ttem_preprocessed['Elevation_Cell']*m_to_ft_factor
    ttem_preprocessed['Elevation_End'] = ttem_preprocessed['Elevation_End']*m_to_ft_factor
    distance_min = ttem_preprocessed['distance'].min()
    elevation_max = ttem_preprocessed['Elevation_Cell'].max()
    elevation_end_max = ttem_preprocessed['Elevation_End'].max()
    ttem_preprocessed['distance_for_plot'] = ttem_preprocessed['distance'] - distance_min
    ttem_preprocessed['Elevation_Cell_for_plot'] = ttem_preprocessed['Elevation_Cell'] - elevation_max
    ttem_preprocessed['Elevation_End_for_plot'] = ttem_preprocessed['Elevation_End'] - elevation_end_max
    ttem_for_plot = abs(ttem_preprocessed[['distance_for_plot', 'Identity_n','Elevation_Cell_for_plot','Elevation_End_for_plot']])
    x_distance = ttem_for_plot['distance_for_plot'].max()
    y_distance = ttem_for_plot['Elevation_End_for_plot'].max()
    empty_grid = np.full((int((y_distance+10)*10),int(x_distance)),np.nan)
# Fill in the tTEM data by loop through the grid
    for index, line in ttem_for_plot.iterrows():
        distance_round = int(line['distance_for_plot'])
        elevation_cell_round = int(line['Elevation_Cell_for_plot'])
        elevation_end_round = int(line['Elevation_End_for_plot'])
        empty_grid[int(elevation_cell_round*10-3*10*m_to_ft_factor):int(elevation_end_round*10+3*10*m_to_ft_factor),int(distance_round):int(distance_round+1)] = line['Identity_n']
    fig = px.imshow(empty_grid, range_color=(1,3))
    fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
    fig.update_layout(
        dict(
        xaxis=dict(
            title='Distance ({})'.format(unit),
            scaleanchor = 'y',
            scaleratio = 10,
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            )
        ),
        yaxis=dict(
            title='Elevation ({})'.format(unit),
            titlefont=dict(
                family="Arial",
                size=50
            ),
            tickfont=dict(
                family="Arial",
                size=45
            ),
            tickmode='array',
            tickvals=list(np.arange(0,empty_grid.shape[0],500)),
            #TODO: this need more modification for m scale
            ticktext=[str(int(elevation_max-i/10)) for i in np.arange(0,empty_grid.shape[0],500)]
        ),
        )
    )
    fig.update_coloraxes(
        colorbar=dict(
            ticks='outside',
            title='Rock Type',
            tickvals=[1,2,3],
            tickfont=dict(
                size=30
            ),
            ticktext=['Fine grained','Mixed Grained','Coarse Grained'],
            tickmode='array',
            len=0.5))
    """fig.update_layout(
        title=dict(
            text='tTEM Line100, Northern Parowan Valley',
        ),
        titlefont=dict(
            size=50
        )
    )"""
    fig.show(renderer='browser')
    return fig, ttem_for_plot, empty_grid

In [2]:
ttem_north = tt.main.ProcessTTEM(ttem_path=[ttemname_north],
                                  welllog=welllog,
                                  DOI_path=DOI)
ttem_north_data = ttem_north.data()
ttem_center = tt.main.ProcessTTEM(ttem_path=[ttemname_center],
                                  welllog=welllog,
                                  DOI_path=DOI)
ttem_center_data = ttem_center.data()

Applying DOI |                                | 69/7968

No layer was excluded
No line was excluded
No point was excluded


Applying DOI |################################| 7968/7968
Applying DOI |                                | 82/7968

No layer was excluded
No line was excluded
No point was excluded


Applying DOI |################################| 7968/7968


In [ ]:
#plot for the paper 20230508 tTEM
ttem_north_data_359 = ttem_north_data[ttem_north_data['ID'] == 359]
plot_gird = np.zeros((int(ttem_north_data_359.Depth_bottom.max()*10),10))
for index, line in ttem_north_data_359.iterrows():
    plot_gird[int(line['Depth_top']*10):int(line['Depth_bottom']*10),:] = np.log10(line['Resistivity'])

In [ ]:
fig = px.imshow(plot_gird, range_color=(0,3),color_continuous_scale=colorRes)
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
fig.update_coloraxes(
    colorbar=dict(
        ticks='outside',
        title='Resistivity',
        tickvals=[0,1,2,3],
        tickfont=dict(
            size=30
        ),
        ticktext=['1','10','100','1000'],
        tickmode='array',
        len=0.5))
"""fig.update_layout(
    title=dict(
        text='tTEM Line100, Northern Parowan Valley',
    ),
    titlefont=dict(
        size=50
    )
)"""
fig.show(renderer='browser')

In [15]:
#Plot for the paper well log 052323
well = tt.process_well.format_well(welllog=welllog,upscale=1)

Formating well logs |################################| 223/223


In [16]:
well_7344 = well[well['Bore'] == '7344']

In [17]:
well_7344

,Bore,Depth1,Depth2,Keyword,Comment,Depth1_m,Depth2_m,Thickness,Elevation,UTMX,UTMY,Elevation_End,Keyword_n
23496,7344,0.0,30.0,fine grain,TOP SOIL,0.000000,1.000000,1.0,1786.705578,347421.286817,4.202192e+06,1785.705578,1
23497,7344,0.0,30.0,fine grain,TOP SOIL,-1.000000,0.000000,1.0,1785.705578,347421.286817,4.202192e+06,1784.705578,1
23498,7344,0.0,30.0,fine grain,TOP SOIL,-2.000000,-1.000000,1.0,1784.705578,347421.286817,4.202192e+06,1783.705578,1
23499,7344,0.0,30.0,fine grain,TOP SOIL,-3.000000,-2.000000,1.0,1783.705578,347421.286817,4.202192e+06,1782.705578,1
23500,7344,0.0,30.0,fine grain,TOP SOIL,-4.000000,-3.000000,1.0,1782.705578,347421.286817,4.202192e+06,1781.705578,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23685,7344,510.0,650.0,coarse grain,BOULDERS,118.440415,119.440415,1.0,1594.265163,347421.286817,4.202192e+06,1593.265163,3
23686,7344,510.0,650.0,coarse grain,BOULDERS,117.440415,118.440415,1.0,1593.265163,347421.286817,4.202192e+06,1592.265163,3
23687,7344,510.0,650.0,coarse grain,BOULDERS,116.440415,117.440415,1.0,1592.265163,347421.286817,4.202192e+06,1591.265163,3
23688,7344,510.0,650.0,coarse grain,BOULDERS,115.440415,116.440415,1.0,1591.265163,347421.286817,4.202192e+06,1590.265163,3


In [6]:
#hist generate 050923
ttem_north_jason = ttem_north_data[ttem_north_data['Line_No'].isin([180])]
ttem_pure_north = ttem_north_data[~ttem_north_data['Line_No'].isin([180])]
hist_north = px.histogram(ttem_pure_north['Resistivity'], nbins=100)
hist_north.update_traces(marker=dict(line=dict(color='black', width=1)))
hist_north.update_xaxes(range=[1,100])
hist_jason = px.histogram(ttem_north_jason['Resistivity'],nbins=100)
hist_jason.update_traces(marker=dict(line=dict(color='black', width=1)))
hist_jason.update_xaxes(range=[1,100])
hist_center = px.histogram(ttem_center_data['Resistivity'])
hist_center.update_traces(marker=dict(line=dict(color='black', width=1)))
hist_center.update_xaxes(range=[1,100])
hist_north.show()
time.sleep(2)
hist_jason.show()
time.sleep(2)
hist_center.show()

KeyboardInterrupt: 

Applying DOI |                                | 86/7968

No layer was excluded
No line was excluded
No point was excluded


Applying DOI |################################| 7968/7968


In [ ]:
#rock trans
Resi_conf_df_publish = pd.DataFrame({'Fine_conf':[8,31],'Mix_conf':[0,0],'Coarse_conf':[31,150]})
ttem_center_data_190 = ttem_center_data[ttem_center_data['Line_No'] == 190]
rk_trans = tt.Rock_trans.rock_transform(ttem_center_data_190,Resi_conf_df_publish)

In [ ]:
block_plot_ID(rk_trans,plot_ft=False)

In [ ]:
#gamma_bradshaw_data = tt.process_gamma.load_gamma(gamma_bradshaw,'GR')
gamma_adams_data = tt.process_gamma.load_gamma(gamma_adams,'GRM')

In [ ]:
gr = tt.main.ProcessGamma(gamma=gamma_halterman,
                              gammaloc=gamma_location,
                              welllog=welllog,
                              rolling=True,
                              window=7,
                              columns="GR",
                              )
output = gr.process()

In [ ]:
gr_ttem = gr.gr_ttem_combine(output,ttem=ttem_center_data)

In [ ]:
gr_ttem

In [ ]:
output

In [ ]:

trace_gr = go.Scatter(x=gr_ttem['GR'],y=gr_ttem['Elevation_Cell']*3.281)
#trace_gr_detail = go.Scatter(x=output['GR'],y=output['Elevation'])
fig_ttem = go.Figure(tt.plot.generate_trace(ttem_north_data,'ttem'))
fig_ttem.show()

In [ ]:
fig.add_trace(trace_gr)
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')

fig.show()

#fig.add_trace(trace_gr_detail)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import YAxis,XAxis,Margin

layout = go.Layout(
    title="",
    xaxis=XAxis(
        title="Resistivity",
        titlefont=dict(
                family="Arial",
                size=50
        ),
        tickfont=dict(
                family="Arial",
                size=40
        )
    ),
    xaxis2 = XAxis(
        overlaying= 'x',
        #dtick = 7,
        title = {'text': 'Gamma count', 'font_family': 'Arial', 'font_size': 50},
        tickfont=dict(
                family="Arial",
                size=40
        ),
        side= 'top'
    ),
    yaxis=dict(
        title="Elevation",
        titlefont=dict(
                family="Arial",
                size=50
        ),
        tickfont=dict(
                family="Arial",
                size=40
    ),
    ))
fig = go.Figure(layout=layout)
fig.add_trace(
    go.Bar(x=gr_ttem['Resistivity'], y=gr_ttem['Elevation_Cell'], name="tTEM_res",orientation = 'h'),
)

fig.add_trace(
    go.Scatter(x=gr_ttem['GRM'], y=gr_ttem['Elevation_Cell'], name="average gamma count", xaxis='x2'),
)
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')

In [ ]:
fig.show()

In [ ]:
avg_gamma = tt.process_gamma.rolling_average(output,window=1000)

In [ ]:
x=170
1786.71 - x/3.218

In [ ]:
avg_gamma